https://huggingface.co/blog/fine-tune-whisper

In [1]:
!pip install --upgrade datasets[audio] transformers accelerate evaluate jiwer tensorboard

ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 324, in run
    session = self.get_default_session(options)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/index_command.py", line 71, in get_default_session
    self._session = self.enter_context(self._build_session(options))
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/index_command.py", line 100, in _build_session
    session = PipSession(
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/network/session.py", line 344, in __init__
    self.headers["User-Agent"] = user_agent()
  File "/usr/local/

In [2]:
from datasets import load_dataset, Audio, DatasetDict, concatenate_datasets
from transformers import WhisperProcessor, WhisperTokenizer, WhisperFeatureExtractor, WhisperForConditionalGeneration
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, pipeline
from huggingface_hub import HfApi, HfFolder, Repository
import torch
from dataclasses import dataclass
import evaluate
from typing import Any, Dict, List, Union
import pandas as pd

# Load Dataset

In [16]:
lang_used=['zh-CN', 'ru-RU', 'fr-FR', 'en-US', 'de-DE']
minds=[]
for i in lang_used:
    minds_data = load_dataset("PolyAI/minds14", i, trust_remote_code=True)
    minds.append(minds_data['train'])

minds = concatenate_datasets(minds)
minds = minds.train_test_split(test_size=0.4, seed = 21, stratify_by_column="intent_class")
minds_test = minds['test'].train_test_split(test_size=0.5)
dataset_dict = DatasetDict({
    'train': minds['train'],
    'test': minds_test['test'],
    'valid': minds_test['train']})

In [ ]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'],
        num_rows: 1652
    })
    test: Dataset({
        features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'],
        num_rows: 551
    })
    valid: Dataset({
        features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'],
        num_rows: 551
    })
})

In [ ]:
dataset_dict["train"][0]

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/28aa727f91fee90575c34956bab09d1716cfaf460c6afcba86a10f04a7d58b83/de-DE~CARD_ISSUES/CA3ee78582d721d7ac7c06a3dd33b035f8_0.wav',
 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/28aa727f91fee90575c34956bab09d1716cfaf460c6afcba86a10f04a7d58b83/de-DE~CARD_ISSUES/CA3ee78582d721d7ac7c06a3dd33b035f8_0.wav',
  'array': array([0.00024414, 0.00048828, 0.00048828, ..., 0.00024414, 0.00024414,
         0.00024414]),
  'sampling_rate': 8000},
 'transcription': 'Hallo kurze frage ich wollte nur wissen warum meine Karte nicht funktioniert hat und ich war heute morgen einkaufen und hat nicht mehr funktioniert',
 'english_transcription': 'Hello short question I just wanted to know why my card did not work and I went shopping this morning and it no longer worked',
 'intent_class': 6,
 'lang_id': 1}

# Data Preparation

In [ ]:
dataset_dict = dataset_dict.remove_columns(["english_transcription", "intent_class", "lang_id"])

In [ ]:
processor = WhisperProcessor.from_pretrained("openai/whisper-tiny", language="english", task="transcribe")

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

In [ ]:
# To simplify using the feature extractor and tokenizer, we can wrap both into a single WhisperProcessor class
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-tiny", language="english", task="transcribe")

input_str = dataset_dict["train"][0]["transcription"]
labels = tokenizer(input_str).input_ids
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}")

Input:                 Hallo kurze frage ich wollte nur wissen warum meine Karte nicht funktioniert hat und ich war heute morgen einkaufen und hat nicht mehr funktioniert
Decoded w/ special:    <|startoftranscript|><|en|><|transcribe|><|notimestamps|>Hallo kurze frage ich wollte nur wissen warum meine Karte nicht funktioniert hat und ich war heute morgen einkaufen und hat nicht mehr funktioniert<|endoftext|>
Decoded w/out special: Hallo kurze frage ich wollte nur wissen warum meine Karte nicht funktioniert hat und ich war heute morgen einkaufen und hat nicht mehr funktioniert
Are equal:             True


In [ ]:
# match the sampling rate of our audio to that of the Whisper model (16kHz)
dataset_dict = dataset_dict.cast_column("audio", Audio(sampling_rate=16000))

In [ ]:
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-tiny")

def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["transcription"]).input_ids
    return batch

In [ ]:
dataset_dict = dataset_dict.map(prepare_dataset)

Map:   0%|          | 0/1652 [00:00<?, ? examples/s]

Map:   0%|          | 0/551 [00:00<?, ? examples/s]

Map:   0%|          | 0/551 [00:00<?, ? examples/s]

In [ ]:
model = WhisperForConditionalGeneration.from_pretrained("artyomboyko/whisper-small-fine_tuned-ru")
model.generation_config.task = "transcribe"

config.json:   0%|          | 0.00/1.31k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.83k [00:00<?, ?B/s]

In [ ]:
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

# Training

In [ ]:
metric = evaluate.load("wer")

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-tiny-multi",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    num_train_epochs=5,
    logging_steps=100,
    gradient_checkpointing=True,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_eval_batch_size=16,
    predict_with_generate=True,
    generation_max_length=256,
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset_dict["train"],
    eval_dataset=dataset_dict["valid"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

In [ ]:
# pre fine tuned performance
trainer.evaluate(dataset_dict["test"])

You have passed task=transcribe, but also have set `forced_decoder_ids` to [[1, None], [2, 50359]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of task=transcribe.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


{'eval_loss': 1.3747094869613647,
 'eval_model_preparation_time': 0.0141,
 'eval_wer': 56.67424027264981,
 'eval_runtime': 397.4556,
 'eval_samples_per_second': 1.386,
 'eval_steps_per_second': 0.088}

In [ ]:
# Start training
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Epoch,Training Loss,Validation Loss,Wer
1,0.492200,0.401710,21.974790
2,0.231900,0.404841,21.036415
3,0.113200,0.417616,20.910364
4,0.060100,0.438259,21.246499
5,0.034000,0.448087,21.288515


You have passed task=transcribe, but also have set `forced_decoder_ids` to [[1, None], [2, 50359]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of task=transcribe.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [], 'begin_suppress_tokens': [220, 50257]}
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version

TrainOutput(global_step=520, training_loss=0.18016424202002013, metrics={'train_runtime': 8423.0609, 'train_samples_per_second': 0.981, 'train_steps_per_second': 0.062, 'total_flos': 2.3837154066432e+18, 'train_loss': 0.18016424202002013, 'epoch': 5.0})

In [ ]:
trainer.evaluate(dataset_dict["test"])

{'eval_loss': 0.41010338068008423,
 'eval_wer': 21.921146953405017,
 'eval_runtime': 417.4099,
 'eval_samples_per_second': 1.32,
 'eval_steps_per_second': 0.084,
 'epoch': 5.0}

In [ ]:
from huggingface_hub import HfApi, HfFolder

api = HfApi()
token = "=================="
HfFolder.save_token(token)

In [ ]:
model.push_to_hub("whisper-small-multilingual")
tokenizer.push_to_hub("whisper-small-multilingual")
processor.push_to_hub("whisper-small-multilingual")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [], 'begin_suppress_tokens': [220, 50257]}


model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/andreanstev/whisper-small-multilingual/commit/3aa22d183e1dfb7f779af44c106b4235f1c97351', commit_message='Upload processor', commit_description='', oid='3aa22d183e1dfb7f779af44c106b4235f1c97351', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Save the model
model.save_pretrained("/content/drive/MyDrive/model/whisper-small-multilingual")

# Save the tokenizer
processor.save_pretrained("/content/drive/MyDrive/model/whisper-small-multilingual")
tokenizer.save_pretrained("/content/drive/MyDrive/model/whisper-small-multilingual")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [], 'begin_suppress_tokens': [220, 50257]}


('/content/drive/MyDrive/model/whisper-small-multilingual/tokenizer_config.json',
 '/content/drive/MyDrive/model/whisper-small-multilingual/special_tokens_map.json',
 '/content/drive/MyDrive/model/whisper-small-multilingual/vocab.json',
 '/content/drive/MyDrive/model/whisper-small-multilingual/merges.txt',
 '/content/drive/MyDrive/model/whisper-small-multilingual/normalizer.json',
 '/content/drive/MyDrive/model/whisper-small-multilingual/added_tokens.json')

In [13]:
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="any", task="transcribe")

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

In [14]:
# pipe = pipeline(model="andreanstev/whisper-small-multilingual", task="automatic-speech-recognition", tokenizer=processor)
pipe = pipeline(model="andreanstev/whisper-small-multilingual", feature_extractor=feature_extractor, tokenizer=tokenizer, device='cuda')

def transcribe(audio):
    text = pipe(audio)["text"]
    return text

In [19]:
example = dataset_dict["test"][22]
text = transcribe(example["audio"]["array"])
print(text)

comment faire un compte et jouer


In [20]:
example["transcription"]

"Bonjour J'aimerais savoir comment créer un compte joint"

In [4]:
example = minds[987]

In [5]:
example

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/28aa727f91fee90575c34956bab09d1716cfaf460c6afcba86a10f04a7d58b83/ru-RU~ADDRESS/6030074c963e11ccd901e5b6.wav',
 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/28aa727f91fee90575c34956bab09d1716cfaf460c6afcba86a10f04a7d58b83/ru-RU~ADDRESS/6030074c963e11ccd901e5b6.wav',
  'array': array([0.00024414, 0.        , 0.00024414, ..., 0.0189209 , 0.03210449,
         0.04187012]),
  'sampling_rate': 8000},
 'transcription': 'Я хотела бы изменить свой адрес',
 'english_transcription': 'I would like to change my address',
 'intent_class': 1,
 'lang_id': 12}

In [15]:
text = transcribe(example["audio"]["array"])
print(text)

could I tell you something
